In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

In [6]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

batch_size = 64

data_dir = '/kaggle/input/satellite-image-classification/data'
data = datasets.ImageFolder(data_dir,transform=transform)

In [7]:
n = len(data)
train_len = int(n*0.8)
test_len = int(n-train_len)

data_train, data_test = random_split(data,[train_len,test_len])

In [8]:
dataloader_train = DataLoader(data_train,batch_size=batch_size,shuffle=True)
dataloader_test = DataLoader(data_test,batch_size=batch_size,shuffle=False)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
num_classes = len(dataloader_train.dataset.dataset.classes)

In [14]:
tuple(dataloader_train.dataset.dataset.classes)

('cloudy', 'desert', 'green_area', 'water')

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Vgg16(nn.Module):
    def __init__(self,num_classes=4):
        super(Vgg16,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,64,3,stride=1,padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64,64,3,stride=1,padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64,128,3,stride=1,padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128,128,3,stride=1,padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(128,256,3,stride=1,padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256,256,3,stride=1,padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256,256,3,stride=1,padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(256,512,3,stride=1,padding=2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512,512,3,stride=1,padding=2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512,512,3,stride=1,padding=2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(512,512,3,stride=1,padding=2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512,512,3,stride=1,padding=2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512,512,3,stride=1,padding=2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer6 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(12*12*512,4096),
            nn.ReLU()
        )
        self.layer7 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU()
        )
        self.layer8 = nn.Sequential(
            nn.Linear(4096,num_classes)
        )

    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
#         print(x.shape)
        x = torch.flatten(x,1)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        return x

In [16]:
model = Vgg16(num_classes).to(device)
epochs=20
learning_rate = 1e-5


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


for epoch in range(epochs):
    for idx,(image,target) in enumerate(dataloader_train):
        images = image.to(device)
        target = target.to(device)

        # forward pass
        y_pred = model(images)
        loss = criterion(y_pred,target)

        # backward pass
        loss.backward()

        # weight pass
        optimizer.step()

        # reset the gradient
        optimizer.zero_grad()
    print(f'epoch number = {epoch}')

epoch number = 0
epoch number = 1
epoch number = 2
epoch number = 3
epoch number = 4
epoch number = 5
epoch number = 6
epoch number = 7
epoch number = 8
epoch number = 9
epoch number = 10
epoch number = 11
epoch number = 12
epoch number = 13
epoch number = 14
epoch number = 15
epoch number = 16
epoch number = 17
epoch number = 18
epoch number = 19


In [17]:
PATH = '/kaggle/working/vgg16C'
torch.save(model.state_dict(), PATH)

In [18]:
model = Vgg16().to(device=device)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [19]:
correct = 0
total = 0
i=0
with torch.no_grad():
    for data in dataloader_test:
        image, label = data
        image = image.to(device)
        label = label.to(device)

        output = model(image)

        _,prediction = torch.max(output.data,1)

        total += label.size(0)
        correct += (prediction==label).sum().item()
        print(i)
        i+=1

print(f'The Accuracy of model is {100*(correct/total)} %')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
The Accuracy of model is 97.95918367346938 %


In [20]:
# prepare to count predictions for each class
classes = tuple(dataloader_train.dataset.dataset.classes)
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
i=0
# again no gradients needed
with torch.no_grad():
    for data in dataloader_test:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1
        
        print(i)
        i+=1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
Accuracy for class: cloudy is 96.9 %
Accuracy for class: desert is 100.0 %
Accuracy for class: green_area is 96.2 %
Accuracy for class: water is 99.7 %
